In [3]:
import pennylane as qml
from pennylane.kernels import  kernel_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
import numpy as np

# ------------------------
# DATA LOADING & PREPROCESSING
# ------------------------
files = ["processed/Autism-Child-Data.csv", "processed/Autism-Adolescent-Data.csv", "processed/Autism-Adult-Data.csv"]
df_list = [pd.read_csv(file) for file in files]
df = pd.concat(df_list, ignore_index=True)

# Cleanup
df = df.dropna().drop_duplicates()

# Convert categorical columns to numerical
categorical_cols = ['gender', 'ethnicity', 'jundice', 'austim', 'contry_of_res', 
                    'used_app_before', 'relation', 'age_desc']

for col in categorical_cols:
    if col in df.columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])

# Binary target
df['Class/ASD'] = df['Class/ASD'].map({'NO': 0, 'YES': 1})

# Features and target
X = df.drop('Class/ASD', axis=1).values
y = df['Class/ASD'].values

print("Combined dataset shape:", df.shape)

# ------------------------
# QUANTUM SVM: QSVM
# ------------------------

# Limit dataset to 10-15 features and max 200-300 samples (quantum practical limit)
n_samples = 100
X = X[:n_samples]
y = y[:n_samples]

# Normalize features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Feature map: Angle Encoding
n_qubits = X.shape[1]
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def kernel_circuit(x1, x2):
    qml.templates.AngleEmbedding(x1, wires=range(n_qubits))
    qml.adjoint(qml.templates.AngleEmbedding)(x2, wires=range(n_qubits))
    return qml.probs(wires=range(n_qubits))

# Kernel function
def quantum_kernel(x1, x2):
    return kernel_circuit(x1, x2)[0]

# Compute kernel matrices
print("Computing quantum kernel matrices...")
X_train_kernel = kernel_matrix(X_train, X_train, kernel=quantum_kernel)
X_test_kernel = kernel_matrix(X_test, X_train, kernel=quantum_kernel)

# Train classical SVM on quantum kernel
clf = SVC(kernel='precomputed')
clf.fit(X_train_kernel, y_train)

# Predict
y_pred = clf.predict(X_test_kernel)

# ------------------------
# RESULTS
# ------------------------
print("\n--- QSVM Results ---")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Combined dataset shape: (947, 21)
Computing quantum kernel matrices...

--- QSVM Results ---
Accuracy: 0.8
              precision    recall  f1-score   support

           0       0.73      1.00      0.85        11
           1       1.00      0.56      0.71         9

    accuracy                           0.80        20
   macro avg       0.87      0.78      0.78        20
weighted avg       0.85      0.80      0.79        20

